# Phase2 - RAG


## About this phase
In this phase you bring in custom data about smorg movies and make your model aware of it using RAG. You store your custom data in an Azure AI Search database, and vectorize the query coming in to find useful results in your custom data using vector search.


If not already done run this in the top level folder:

```
pip install -r requirements.txt
```




In [1]:
import os
from dotenv import load_dotenv

# Load environment variables
if load_dotenv():
    print("Found Azure OpenAI API Base Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else: 
    print("Azure OpenAI API Base Endpoint not found. Have you configured the .env file?")

Found Azure OpenAI API Base Endpoint: https://cog-zfeqxabh5ia2g.openai.azure.com/


# Create vector search index

Create your search index schema and vector search configuration

In [2]:
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential

from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch,
    SearchIndex

)

credential = AzureKeyCredential(os.environ["AZURE_AI_SEARCH_KEY"]) if len(os.environ["AZURE_AI_SEARCH_KEY"]) > 0 else DefaultAzureCredential()

index_name = "movies-semantic-index"

index_client = SearchIndexClient(
    endpoint=os.environ["AZURE_AI_SEARCH_ENDPOINT"], 
    credential=credential
)

# Create a search index with the fields and a vector field which we will fill with a vector based on the overview field
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="genre", type=SearchFieldDataType.String),
    SearchableField(name="title", type=SearchFieldDataType.String),
    SearchableField(name="year", type=SearchFieldDataType.String),
    SearchableField(name="rating", type=SearchFieldDataType.String),
    SearchableField(name="plot", type=SearchFieldDataType.String),
    SearchField(name="vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_profile_name="myHnswProfile"),
]

# Configure the vector search configuration  
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="myHnsw"
        )
    ],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
        )
    ]
)

# Configure the semantic search configuration to prefer title and tagline fields over overview
semantic_config = SemanticConfiguration(
    name="movies-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="title"),
        keywords_fields=[SemanticField(field_name="genre")],
        content_fields=[SemanticField(field_name="plot")]
    )
)

# Create the semantic settings with the configuration
semantic_search = SemanticSearch(configurations=[semantic_config])

# Create the search index with the semantic settings
index = SearchIndex(name=index_name, fields=fields,
                    vector_search=vector_search, semantic_search=semantic_search)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')

 movies-semantic-index created


This is the object model for receiving questions.

In [3]:
from enum import Enum
from pydantic import BaseModel

class QuestionType(str, Enum):
    multiple_choice = "multiple_choice"
    true_or_false = "true_or_false"
    estimation = "estimation"

class Ask(BaseModel):
    question: str | None = None
    type: QuestionType
    correlationToken: str | None = None

class Answer(BaseModel):
    answer: str
    correlationToken: str | None = None
    promptTokensUsed: int | None = None
    completionTokensUsed: int | None = None


## Load custom data
Here's how you load custom vector data by means of an embedding model and then query it.


In [4]:
import os
import json
from openai import AzureOpenAI
from azure.search.documents import SearchClient

client = AzureOpenAI(
        api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
        api_version = os.getenv("AZURE_OPENAI_VERSION"),
        azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

# use an embeddingsmodel to create embeddings
def get_embedding(text, model=os.getenv("AZURE_OPENAI_EMBEDDING_MODEL")):
    return client.embeddings.create(input = [text], model=model).data[0].embedding

# 1. define function to parse csv row and create embedding for overview text
def parseMovie(movie):
    print(movie)
    return dict([
        ("id", str(movie["movie_id"])),
        ("genre", movie["movie_genre"]),
        ("title", movie["movie_title"]),
        ("year", str(movie["movie_year"])),
        ("rating", str(movie["movie_rating"])),
        ("plot", movie["movie_plot"]),
        ("vector", get_embedding(movie["movie_plot"]))
    ])

# 2. load movies from csv
movies = []
with open('./movies.json') as json_data:
    moviesJson = json.load(json_data)
    line_count = 0
    for movieJson in moviesJson:
        movieEmbedding = parseMovie(movieJson)
        movies.append(movieEmbedding)
        line_count += 1
    print(f'Processed {line_count} lines.')
print('Loaded %s movies.' % len(movies))


# 3. upload documents to vector store
search_client = SearchClient(
    endpoint=os.environ["AZURE_AI_SEARCH_ENDPOINT"], 
    index_name=index_name,
    credential=credential
)

result = search_client.upload_documents(movies)
print(f"Successfully loaded {len(movies)} movies into Azure AI Search index.")



{'movie_id': 1, 'movie_genre': 'Action', 'movie_title': 'The Smonger Games', 'movie_year': 2026, 'movie_rating': 8, 'movie_plot': "In the alien planet Smorgia, a group of Smoorghs are forced to compete in a deadly game called The Smonger Games. The games are organized by the tyrannical ruler of Smorgia, who enjoys watching the competition. Each participant has a unique special ability, and the last survivor will be granted their greatest desires. As the games progress, alliances are formed, betrayals occur, and the true nature of the ruler is revealed. The main character, Smok, discovers that his special ability allows him to control fire. With this power, he becomes a formidable contender and starts a rebellion against the ruler. Smok leads a group of Smoorghs to fight for their freedom and overthrow the oppressive regime. The movie was filmed in the stunning city of Smonopolis and had a budget of 50 million Smorps.The Smonger Games featured 20 actors from different planets, making it

RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the Embeddings_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 6 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}

# Query index and create a response

In [5]:
from openai import AzureOpenAI
from azure.search.documents.models import (
    VectorizedQuery
)

client = AzureOpenAI(
        api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
        api_version = os.getenv("AZURE_OPENAI_VERSION"),
        azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

deployment_name = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
model_name = os.getenv("AZURE_OPENAI_COMPLETION_MODEL")

index_client = SearchClient(
    endpoint=os.environ["AZURE_AI_SEARCH_ENDPOINT"], 
    index_name=index_name,
    credential=credential
)

question = "Tell me about the latest Ant Man movie. When was it released?"

# create a vectorized query based on the question
vector = VectorizedQuery(vector=get_embedding(question), k_nearest_neighbors=5, fields="vector")


# create search client to retrieve movies from the vector store
found_docs = list(search_client.search(
    search_text=None,
    query_type="semantic",
    semantic_configuration_name="movies-semantic-config",
    vector_queries=[vector],
    select=["title", "genre", "plot", "year"],
    top=5
))

found_docs_as_text = " "
# print the found documents and the field that were selected
for doc in found_docs:
    print("Movie: {}".format(doc["title"]))
    print("Genre: {}".format(doc["genre"]))
    print("Year: {}".format(doc["year"]))
    print("----------")
    found_docs_as_text += " "+ "Movie Title: {}".format(doc["title"]) +" "+ "Release Year: {}".format(doc["year"]) + " "+ "Movie Plot: {}".format(doc["plot"])
    
# augment the question with the found documents and ask the LLM to generate a response
system_prompt = "Here is what you need to do:"

parameters = [system_prompt, ' Context:', found_docs_as_text , ' Question:', question]
joined_parameters = ''.join(parameters)

response = client.chat.completions.create(
        model = deployment_name,
        messages = [{"role" : "assistant", "content" : joined_parameters}],
    )

print (response.choices[0].message.content)

NameError: name 'search_client' is not defined

# YOUR Mission: 
Adjust the function below and reuse it in the main.py file later to deploy to Azure and to update your service. 
Ensure the answers provided are correct and in the correct format.



In [ ]:

async def ask_question(ask: Ask):
    """
    Ask a question
    """
    
    start_phrase =  ask.question
    response: openai.types.chat.chat_completion.ChatCompletion = None

    #####\n",
    # implement rag flow here\n",
    ######\n",
   
    answer = Answer(answer=response.choices[0].message.content)
    answer.correlationToken = ask.correlationToken
    answer.promptTokensUsed = response.usage.prompt_tokens
    answer.completionTokensUsed = response.usage.completion_tokens
    
    return answer

Use this snippet to try your method with several questions.

In [ ]:

ask = Ask(question="How many actors were featured in The Smonger Games?", type=QuestionType.estimation)
answer = await ask_question(ask)
print('Answer:', answer)

Make sure you transfer your code changes into main.py (or additional files). Then redeploy your container using this command.
```
bash ./azd-hooks/deploy.sh phase2 $AZURE_ENV_NAME
```
